In [1]:
import pandas as pd

In [2]:
import itertools

In [3]:
import glob

In [4]:
import warnings
warnings.filterwarnings('ignore')

In [5]:
import matplotlib.pyplot as plt

import numpy as np

In [6]:
def prepare_data(dimensions):
    
    columns = (['player index', 'co player index'] + [f'p{i}' for i in range(1, 17)] + 
               [f'q{i}' for i in range(1, 17)] + ['e', 'cc', 'cd', 'dc', 'dd'])
    
    files = glob.glob(f"../src/evolutionary_simulations/cooperation_data/memone/error_0.01_n_{dimensions}_*.csv")
    
#     files = glob.glob(f"../src/evolutionary_simulations/cooperation_data/error_0.01_sdim_{dimensions}_*")
    
    dfs = [pd.read_csv(file, header=None, names=columns) for file in files]

    df = pd.concat(dfs)
    
    rhos = df[df['player index'] == df['co player index']]
    rhos['rho'] = 0
    for i, row in rhos.iterrows():
        rhos.loc[i, 'rho'] = row['cc'] + row['cd']
    df = df.merge(rhos[['player index', 'rho']], how='left', left_on='player index', right_on='player index')
    
    
    df['rho p'] = df.apply(lambda row: row['cc'] + row['cd'], axis=1)

    df['rho q'] = df.apply(lambda row: row['cc'] + row['dc'], axis=1)

    df['x'] = df.apply(lambda row: row['rho'] - row['rho p'], axis=1)

    df['y'] = df.apply(lambda row: row['rho'] - row['rho q'], axis=1)
    
    df['y / x'] = df['y'] / df['x']
    
    return df

In [7]:
def vaquero_method(df):
    
    sets = []

    for j, group in df.groupby('player index'):

        Q1, Q2, Q3, Q4 = [], [], [], []

        for i, row in group.iterrows():

            if row['player index'] != row['co player index']:

                if row['x'] > 0 and row['y'] > 0:
                    Q1.append(row['y'] / row['x'])

                if row['x'] < 0 and row['y'] <= 0:

                    Q2.append(row['y'] / row['x'])

                if row['x'] <= 0 and row['y'] > 0:
                    Q3.append(row['co player index'])

                if row['x'] == 0 and row['y'] == 0:

                    Q4.append(row['co player index'])

        sets.append([j, Q1, Q2, Q3, Q4])
    return sets

In [8]:
def payoffs_of_p(rho, b, c):
    return (b * rho) - (c * rho)

def payoffs_of_q(rho_p, rho_q, b, c):
    return (b * rho_p) - (c * rho_q)

### Mem two

In [9]:
memory_two = prepare_data(2)

In [10]:
sets = vaquero_method(memory_two)

In [11]:
df = pd.DataFrame(sets, columns=['player index', 'Q1 (y / x)', 'Q2 (y / x)', 'Q3 (indices)', 'Q4 (indices)'])

In [13]:
df

,player index,Q1 (y / x),Q2 (y / x),Q3 (indices),Q4 (indices)
0,32778,"[1.525847282789215, 1.5231784085722515, 1.0069...",[],[],[]
1,33050,"[2.041128718901453, 2.04102469438476, 1.015438...","[105.32258073180721, 105.33225815197075, 105.3...",[],[]
2,34954,"[1.5258995892031346, 1.5232304580751057, 1.010...",[],[],[]
3,35226,"[2.0413337990593474, 2.041230734887535, 1.0207...",[],[],[]
4,36874,"[1.5258220494870627, 1.5258220475233837, 1.007...",[],[],[]
5,36880,"[2.077575666558791, 1.552157121421291, 2.07216...",[],[],[]
6,37146,"[2.0411287047516518, 2.0411287061466945, 1.015...",[],[],[]
7,39050,"[1.5258486034981635, 1.525848602175977, 1.0104...",[],[],[]
8,39322,"[2.0412328196584757, 2.0412328175008234, 1.020...",[],[],[]
9,61450,"[1.515517874438259, 2.010959696892494, 1.01031...",[],[],[]


In [14]:
Q3_empty = []
Q4_empty = []

for i, row in df.iterrows():
    Q3_empty.append(len(row['Q3 (indices)']) == 0)
    Q4_empty.append(len(row['Q4 (indices)']) == 0)

In [15]:
df['Q3 empty'] = Q3_empty
df['Q4 empty'] = Q4_empty

In [20]:
order_from_paper = [36874,32778,34954,39050,39322,35226,33050,37146,36880,61450,61456]

sorterIndex = dict(zip(order_from_paper, range(len(order_from_paper))))

In [23]:
df['player index'] = df['player index'].map(sorterIndex)

In [26]:
df = df.sort_values('player index')

In [27]:
for i, row in df[(df['Q3 empty'] == True) & (df['Q4 empty'] == True)].iterrows():
    ub = False
    lb = False
    
    if row['Q1 (y / x)']:
        lb = max(row['Q1 (y / x)'])
    if row['Q2 (y / x)']:
        ub = min(row['Q2 (y / x)'])
        
#     if lb <= ub:
    
    print(f"{row['player index']}: {round(lb, 2)} <= b/c <= {round(ub, 2)}")

0: 1.53 <= b/c <= 0
1: 1.53 <= b/c <= 0
2: 1.53 <= b/c <= 0
3: 1.53 <= b/c <= 0
4: 2.04 <= b/c <= 0
5: 2.04 <= b/c <= 0
6: 2.04 <= b/c <= 105.32
7: 2.04 <= b/c <= 0
8: 2.08 <= b/c <= 0
9: 2.02 <= b/c <= 0
10: 2.04 <= b/c <= 0


### One bit

In [9]:
one_bit = prepare_data(2)

In [10]:
sets = vaquero_method(one_bit)

In [11]:
pd.DataFrame(sets, columns=['player index', 'Q1 (y / x)', 'Q2 (y / x)', 'Q3 (indices)', 'Q4 (indices)'])

,player index,Q1 (y / x),Q2 (y / x),Q3 (indices),Q4 (indices)
0,1,[],[],[],[]
1,2,[],[],[1.0],[3.0]
2,3,[1.0204081632653061],[1.0204081632653061],[],[2.0]
3,4,[],[],"[1.0, 2.0, 3.0]",[]


In [12]:
one_bit[one_bit['player index'] == 1]

,player index,co player index,p1,p2,q1,q2,e,cc,cd,dc,dd,rho,rho p,rho q,x,y,y / x
0,1,1,0,0,0,0,0.01,0.000100,0.009900,0.009900,0.980100,0.01,0.01,0.0100,0.0,0.0000,NaN
1,1,2,0,0,0,1,0.01,0.009802,0.000198,0.970398,0.019602,0.01,0.01,0.9802,0.0,-0.9702,-inf
2,1,3,0,0,1,0,0.01,0.000198,0.009802,0.019602,0.970398,0.01,0.01,0.0198,0.0,-0.0098,-inf
3,1,4,0,0,1,1,0.01,0.009900,0.000100,0.980100,0.009900,0.01,0.01,0.9900,0.0,-0.9800,-inf


### Two bits

In [13]:
two_bit = prepare_data(4)

In [14]:
sets = vaquero_method(two_bit)

In [15]:
df = pd.DataFrame(sets, columns=['player index', 'Q1 (y / x)', 'Q2 (y / x)', 'Q3 (indices)', 'Q4 (indices)'])


In [16]:
Q3_empty = []
Q4_empty = []

for i, row in df.iterrows():
    Q3_empty.append(len(row['Q3 (indices)']) == 0)
    Q4_empty.append(len(row['Q4 (indices)']) == 0)

In [17]:
df['Q3 empty'] = Q3_empty
df['Q4 empty'] = Q4_empty

In [18]:
df[(df['Q3 empty'] == True) & (df['Q4 empty'] == True)]

,player index,Q1 (y / x),Q2 (y / x),Q3 (indices),Q4 (indices),Q3 empty,Q4 empty
0,1,[],[],[],[],True,True
2,3,"[1.041232819593638, 1.0, 0.7928423310639318, 1...",[],[],[],True,True
4,5,"[1.0412328196234282, 1.0, 1.0412123626241727]",[2882353642.7057486],[],[],True,True
8,9,[19.503333337744937],"[1.5152617371163994, 51.897825271308754, 1.361...",[],[],True,True
14,15,"[1.020306234430547, 1.3437206210817625, 1.0494...",[19.503623311907123],[],[],True,True


In [19]:
two_bit[two_bit['player index'] == 15]

,player index,co player index,p1,p2,p3,p4,q1,q2,q3,q4,...,cc,cd,dc,dd,rho,rho p,rho q,x,y,y / x
240,15,1,1,1,1,0,0,0,0,0,...,0.000295,0.029207,0.009705,0.960793,0.989897,0.029502,0.010000,0.960395,0.979897,1.020306
241,15,2,1,1,1,0,0,0,0,1,...,0.166435,0.334463,0.166384,0.332718,0.989897,0.500898,0.332819,0.488999,0.657078,1.343721
242,15,3,1,1,1,0,0,0,1,0,...,0.018621,0.063206,0.018345,0.899828,0.989897,0.081827,0.036966,0.908070,0.952931,1.049402
243,15,4,1,1,1,0,0,0,1,1,...,0.281692,0.293876,0.144251,0.280181,0.989897,0.575568,0.425943,0.414329,0.563953,1.361126
244,15,5,1,1,1,0,0,1,0,0,...,0.008886,0.441635,0.218089,0.331390,0.989897,0.450520,0.226975,0.539376,0.762922,1.414452
245,15,6,1,1,1,0,0,1,0,1,...,0.202428,0.398745,0.198422,0.200405,0.989897,0.601173,0.400850,0.388723,0.589047,1.515337
246,15,7,1,1,1,0,0,1,1,0,...,0.164833,0.452118,0.153722,0.229327,0.989897,0.616951,0.318555,0.372946,0.671342,1.800103
247,15,8,1,1,1,0,0,1,1,1,...,0.334463,0.332718,0.166435,0.166384,0.989897,0.667181,0.500898,0.322716,0.488999,1.515262
248,15,9,1,1,1,0,1,0,0,0,...,0.327875,0.230201,0.114049,0.327875,0.989897,0.558076,0.441924,0.431821,0.547973,1.268982
249,15,10,1,1,1,0,1,0,0,1,...,0.647747,0.230857,0.115554,0.005841,0.989897,0.878605,0.763301,0.111292,0.226595,2.036042


In [20]:
2 ** 4

16

In [22]:
for i, row in df[(df['Q3 empty'] == True) & (df['Q4 empty'] == True)].iterrows():
    ub = False
    lb = False
    
    if row['Q1 (y / x)']:
        lb = max(row['Q1 (y / x)'])
    if row['Q2 (y / x)']:
        ub = min(row['Q2 (y / x)'])
        
#     if lb <= ub:
    
    print(f"{row['player index']}: {round(lb, 2)} <= b/c <= {round(ub, 2)}")

1: 0 <= b/c <= 0
3: 1.04 <= b/c <= 0
5: 1.04 <= b/c <= 2882353642.71
9: 19.5 <= b/c <= 1.02
15: 51.9 <= b/c <= 19.5


In [44]:
index = 9
df[(df['player index'] == index)].round(3)

,player index,Q1 (y / x),Q2 (y / x),Q3 (indices),Q4 (indices),Q3 empty,Q4 empty
8,9,[19.503333337744937],"[1.5152617371163994, 51.897825271308754, 1.361...",[],[],True,True


In [68]:
index = 3
two_bit[(two_bit['player index'] == index) & (two_bit['co player index'] == index)].iloc[0]

player index       3.000000
co player index    3.000000
p1                 0.000000
p2                 0.000000
p3                 1.000000
p4                 0.000000
q1                 0.000000
q2                 0.000000
q3                 1.000000
q4                 0.000000
e                  0.010000
cc                 0.005048
cd                 0.249952
dc                 0.249952
dd                 0.495048
rho                0.255000
rho p              0.255000
rho q              0.255000
x                  0.000000
y                  0.000000
y / x                   NaN
Name: 50, dtype: float64

In [66]:
index = 3
row = two_bit[(two_bit['player index'] == index) & (two_bit['co player index'] == 14)].iloc[0]

In [67]:
payoffs_of_p(row['rho'], b=row['y / x'] + 0.1, c=1)

-148749997.85548145

In [42]:
payoffs_of_q(row['rho p'], row['rho q'], b=row['y / x'] + 0.1, c=1)

735000179.2369367

In [27]:
two_bit[(two_bit['player index'] == index)]

,player index,co player index,p1,p2,p3,p4,q1,q2,q3,q4,...,cc,cd,dc,dd,rho,rho p,rho q,x,y,y / x
96,5,1,0,1,0,0,0,0,0,0,...,0.000197,0.019505,0.009803,0.970495,0.255,0.019702,0.010000,2.352980e-01,2.450000e-01,1.041233e+00
97,5,2,0,1,0,0,0,0,0,1,...,0.018345,0.018621,0.899828,0.063206,0.255,0.036966,0.918173,2.180341e-01,-6.631734e-01,-3.041604e+00
98,5,3,0,1,0,0,0,0,1,0,...,0.007355,0.247645,0.247645,0.497355,0.255,0.255000,0.255000,6.700001e-10,6.700001e-10,1.000000e+00
99,5,4,0,1,0,0,0,0,1,1,...,0.243069,0.011931,0.497031,0.247969,0.255,0.255000,0.740100,6.000000e-10,-4.851000e-01,-8.084999e+08
100,5,5,0,1,0,0,0,1,0,0,...,0.085162,0.169838,0.169838,0.575162,0.255,0.255000,0.255000,0.000000e+00,0.000000e+00,NaN
101,5,6,0,1,0,0,0,1,0,1,...,0.247550,0.007450,0.492550,0.252450,0.255,0.255000,0.740100,2.000000e-10,-4.851000e-01,-2.425500e+09
102,5,7,0,1,0,0,0,1,1,0,...,0.012989,0.216344,0.437710,0.332957,0.255,0.229333,0.450699,2.566690e-02,-1.956990e-01,-7.624568e+00
103,5,8,0,1,0,0,0,1,1,1,...,0.019769,0.000207,0.969937,0.010086,0.255,0.019977,0.989706,2.350233e-01,-7.347065e-01,-3.126100e+00
104,5,9,0,1,0,0,1,0,0,0,...,0.000205,0.019775,0.010089,0.969932,0.255,0.019979,0.010294,2.350206e-01,2.447064e-01,1.041212e+00
105,5,10,0,1,0,0,1,0,0,1,...,0.018942,0.018851,0.898269,0.063938,0.255,0.037793,0.917211,2.172067e-01,-6.622107e-01,-3.048758e+00


In [35]:
max(df[df['player index'] == 15]['Q1 (y / x)'])

[1.020306234430547,
 1.3437206210817625,
 1.0494022874432454,
 1.3611263417362822,
 1.4144515802339694,
 1.515336844350368,
 1.8001032943045303,
 1.515261735387235,
 1.2689824441476485,
 2.0360422521213506,
 15.738157673674284,
 51.8979376368892,
 32.46112270446797,
 51.89766379830998]

In [43]:
row['rho'], row['rho p'], row['rho q']

(253    0.989897
 Name: rho, dtype: float64,
 253    0.989706
 Name: rho p, dtype: float64,
 253    0.980021
 Name: rho q, dtype: float64)

253    1.979793
Name: rho, dtype: float64

In [49]:
payoffs_of_q(row['rho p'], row['rho q'], b=3, c=1)

253    1.989099
dtype: float64

In [46]:
row

,player index,co player index,p1,p2,p3,p4,q1,q2,q3,q4,...,cc,cd,dc,dd,rho,rho p,rho q,x,y,y / x
253,15,14,1,1,1,0,1,1,0,1,...,0.969932,0.019775,0.010089,0.000205,0.989897,0.989706,0.980021,0.00019,0.009876,51.897664


In [108]:
row['rho'] - row['rho p']

109   -1.700000e-10
dtype: float64

In [111]:
row['rho'] - row['rho q']

109   -0.49
dtype: float64

In [114]:
row['y']

109   -0.49
Name: y, dtype: float64

In [93]:
payoffs_of_p(0.5, 2, 1)

0.5

In [94]:
payoffs_of_q(0.5, 0.2, 2, 1)

0.8

In [91]:
payoffs_of_q(row['rho p'].iloc[0], row['rho q'].iloc[0], b=row['y / x'].iloc[0] - 0.001, c=1)

735000179.2111816

In [77]:
row['rho'].iloc[0] * row['y / x'].iloc[0] - row['rho'].iloc[0] * 1

735000179.2114366

In [79]:
row['rho p'].iloc[0] * row['y / x'].iloc[0] - row['rho q'].iloc[0] * 1

735000179.2114366

In [89]:
row['rho p'].iloc[0] * 2 - row['rho q'].iloc[0], row['rho'].iloc[0] * 2 - row['rho'].iloc[0]

(-0.23499999926000004, 0.2550000002)

In [26]:
index = 5

two_bit[(two_bit['player index'] == index) & (two_bit['co player index'] == index)].iloc[0][2:6]

p1    0.0
p2    1.0
p3    0.0
p4    0.0
Name: 100, dtype: float64

### Three bits

In [21]:
three_bit = prepare_data(8)

In [22]:
three_bit

,player index,co player index,p1,p2,p3,p4,p5,p6,p7,p8,...,cc,cd,dc,dd,rho,rho p,rho q,x,y,y / x
0,157,1,1,0,0,1,1,1,0,0,...,0.000293,0.029015,0.009707,0.960985,0.503265,0.029308,0.010000,0.473957,0.493265,1.040738
1,157,2,1,0,0,1,1,1,0,0,...,0.111159,0.226630,0.217875,0.444335,0.503265,0.337790,0.329035,0.165475,0.174230,1.052908
2,157,3,1,0,0,1,1,1,0,0,...,0.017460,0.060853,0.018306,0.903381,0.503265,0.078313,0.035766,0.424952,0.467499,1.100121
3,157,4,1,0,0,1,1,1,0,0,...,0.193975,0.209113,0.198761,0.398151,0.503265,0.403088,0.392736,0.100177,0.110529,1.103332
4,157,5,1,0,0,1,1,1,0,0,...,0.015943,0.045318,0.018380,0.920359,0.503265,0.061261,0.034323,0.442005,0.468942,1.060945
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
65531,27,252,0,0,0,1,1,0,1,0,...,0.134980,0.003924,0.730952,0.130144,0.297594,0.138904,0.865931,0.158690,-0.568338,-3.581438
65532,27,253,0,0,0,1,1,0,1,0,...,0.230394,0.140705,0.371234,0.257667,0.297594,0.371099,0.601628,-0.073505,-0.304035,4.136220
65533,27,254,0,0,0,1,1,0,1,0,...,0.172667,0.006500,0.654679,0.166154,0.297594,0.179166,0.827346,0.118427,-0.529752,-4.473231
65534,27,255,0,0,0,1,1,0,1,0,...,0.209989,0.105719,0.425460,0.258832,0.297594,0.315708,0.635449,-0.018114,-0.337855,18.651632


In [127]:
three_bit = three_bit.sort_values('player index')

In [128]:
sets = vaquero_method(three_bit)

In [129]:
df = pd.DataFrame(sets, columns=['player index', 'Q1 (y / x)', 'Q2 (y / x)', 'Q3 (indices)', 'Q4 (indices)'])

In [130]:
Q3_empty = []
Q4_empty = []

for i, row in df.iterrows():
    Q3_empty.append(len(row['Q3 (indices)']) == 0)
    Q4_empty.append(len(row['Q4 (indices)']) == 0)

In [131]:
df['Q3 empty'] = Q3_empty
df['Q4 empty'] = Q4_empty

In [132]:
df[(df['Q3 empty'] == True) & (df['Q4 empty'] == True)]

,player index,Q1 (y / x),Q2 (y / x),Q3 (indices),Q4 (indices),Q3 empty,Q4 empty
0,1,[],[],[],[],True,True
2,3,"[2.1001929949428306, 2.1358696922160805, 2.113...","[4.426272475846901, 1.0454759057798695, 6.8511...",[],[],True,True
4,5,"[0.7878532603527855, 0.35133089983325466, 0.82...","[1.0918229259155434, 13.075507275722478, 1.160...",[],[],True,True
8,9,"[52.30969477049199, 52.04859561111178]","[4.522248449893896, 35.47163558736799, 4.85328...",[],[],True,True
10,11,"[2.1156963708681378, 2.1165644709830227, 2.038...","[5.485695387655202, 8.662950857996368, 4.25870...",[],[],True,True
12,13,"[0.9586607792381594, 0.6226693890079458, 0.999...","[5.902087850346785, 1927733381.6056175, 5.0212...",[],[],True,True
16,17,"[0.8124763937952045, 1.0626196093008242, 1.062...","[1.0633465854616213, 13.633630504273109, 1.186...",[],[],True,True
24,25,"[1.061551115358093, 0.3924669546862254, 14.967...","[4.95780694927603, 4.56185495958249, 2.0824016...",[],[],True,True
32,33,"[2.016413547104868, 0.9999052185132165, 1.0099...","[5.995964905399071, 5.945628120440939, 5.00472...",[],[],True,True
34,35,"[1.041232819593638, 0.7135921390125219, 0.9554...","[401.6827721781851, 1.817913203581593, 1.37384...",[],[],True,True


In [133]:
for i, row in df[(df['Q3 empty'] == True) & (df['Q4 empty'] == True)].iterrows():
    ub = False
    lb = False
    
    if row['Q1 (y / x)']:
        lb = max(row['Q1 (y / x)'])
    if row['Q2 (y / x)']:
        ub = min(row['Q2 (y / x)'])
    if lb <= ub:
    
        print(f"{row['player index']}: {lb} <= b/c <= {ub}")

1: False <= b/c <= False
35: 1.0412328199480185 <= b/c <= 1.0424536300792253
69: 1.0412328408761917 <= b/c <= 1.0436048213156999


In [108]:
index = 35

three_bit[(three_bit['player index'] == index) & (three_bit['co player index'] == index)]

,player index,co player index,p1,p2,p3,p4,p5,p6,p7,p8,...,cc,cd,dc,dd,rho,rho p,rho q,x,y,y / x
14242,35,35,0,0,1,0,0,0,1,0,...,0.005048,0.249952,0.249952,0.495048,0.255,0.255,0.255,0.0,0.0,NaN


In [54]:
index = 35

three_bit[(three_bit['player index'] == index) & (three_bit['co player index'] == index)].iloc[0][2:]

p1       0.000000
p2       0.000000
p3       1.000000
p4       0.000000
p5       0.000000
p6       0.000000
p7       1.000000
p8       0.000000
q1       0.000000
q2       0.000000
q3       1.000000
q4       0.000000
q5       0.000000
q6       0.000000
q7       1.000000
q8       0.000000
e        0.010000
cc       0.005048
cd       0.249952
dc       0.249952
dd       0.495048
rho      0.255000
rho p    0.255000
rho q    0.255000
x        0.000000
y        0.000000
y / x         NaN
Name: 3426, dtype: float64

In [55]:
import itertools

In [56]:
list(itertools.product(['C', 'D'], repeat=3))

[('C', 'C', 'C'),
 ('C', 'C', 'D'),
 ('C', 'D', 'C'),
 ('C', 'D', 'D'),
 ('D', 'C', 'C'),
 ('D', 'C', 'D'),
 ('D', 'D', 'C'),
 ('D', 'D', 'D')]

### Four bits

In [57]:
four_bit = prepare_data(16)

In [58]:
sets = vaquero_method(four_bit)

In [59]:
df = pd.DataFrame(sets, columns=['player index', 'Q1 (y / x)', 'Q2 (y / x)', 'Q3 (indices)', 'Q4 (indices)'])

In [60]:
Q3_empty = []
Q4_empty = []

for i, row in df.iterrows():
    Q3_empty.append(len(row['Q3 (indices)']) == 0)
    Q4_empty.append(len(row['Q4 (indices)']) == 0)

In [61]:
df['Q3 empty'] = Q3_empty
df['Q4 empty'] = Q4_empty

In [62]:
df[(df['Q3 empty'] == True) & (df['Q4 empty'] == True)]

,player index,Q1 (y / x),Q2 (y / x),Q3 (indices),Q4 (indices),Q3 empty,Q4 empty
0,1,[],"[470691842700.7444, 4754463058.078226, 4754463...",[],[],True,True
2,3,"[2.147340278653103, 2.1351953799526076, 2.1356...","[5.693177909845503, 8.232698998023187, 5.76735...",[],[],True,True
4,5,"[1.0847249330974662, 1.0866014890150115, 0.760...","[1.0830785947030641, 0.9748031769471045, 4.328...",[],[],True,True
8,9,"[53.351602927097574, 53.35170801682954]","[5.995437172999901, 53.08052068238932, 6.97652...",[],[],True,True
10,11,"[2.1244617651580415, 4.13316579705317, 2.10199...","[1.0704277555715214, 2.0948818659426056, 4.170...",[],[],True,True
12,13,"[1.0625931095255743, 1.0625931091050795, 1.062...","[14.241719468353864, 7.3041736193454065, 14.11...",[],[],True,True
14,15,"[1.6139154900415313, 2.2220863658042926, 3.019...","[0.9939779892705667, 1.6719006289802911, 1.706...",[],[],True,True
16,17,"[1.0818203313761483, 0.9985374769968246, 0.765...","[0.850970862731302, 1.2414917367178695, 1.1082...",[],[],True,True
24,25,"[1.0652778637777383, 1.1313037772823442, 1.085...","[14.108403244944654, 0.9961983476474952, 74.07...",[],[],True,True
32,33,"[2.0420346590887632, 1.0102098714812207, 0.999...","[10661.733062446496, 36.54546559617723, 10252....",[],[],True,True


In [65]:
indices = []
lbounds = []
ubounds = []
for i, row in df[(df['Q3 empty'] == True) & (df['Q4 empty'] == True)].iterrows():
    ub = False
    lb = False
    
    if row['Q1 (y / x)']:
        lb = max(row['Q1 (y / x)'])
    if row['Q2 (y / x)']:
        ub = min(row['Q2 (y / x)'])
        
    if lb <= ub:
    
        print(f"{row['player index']}: {lb} <= b/c <= {ub}", f"-{lb <= ub}")
        indices.append(row['player index'])
        lbounds.append(round(lb, 3))
        ubounds.append(round(ub, 3))

1: False <= b/c <= 485101.21231723594 -True
35: 1.0414495046715793 <= b/c <= 1.0415829926006013 -True
73: 1.0419170119230037 <= b/c <= 1.047882076486478 -True
163: 1.0413538739633985 <= b/c <= 1.0415574235985448 -True
205: 1.0412328937089776 <= b/c <= 1.0426671633511733 -True
209: 1.0867506256938408 <= b/c <= 1.789273796073751 -True


In [181]:
columns = [f'p{i}' for i in range(1, 17)] + ['rho']

table = four_bit[(four_bit['player index'].isin(indices))][columns].drop_duplicates().reset_index(drop=True).round(3)

In [183]:
table['lb'] = lbounds
table['ub'] = ubounds

In [186]:
table.to_latex()

'\\begin{tabular}{lrrrrrrrrrrrrrrrrrrr}\n\\toprule\n{} &  p1 &  p2 &  p3 &  p4 &  p5 &  p6 &  p7 &  p8 &  p9 &  p10 &  p11 &  p12 &  p13 &  p14 &  p15 &  p16 &    rho &     lb &          ub \\\\\n\\midrule\n0 &   0 &   0 &   0 &   0 &   0 &   0 &   0 &   0 &   0 &    0 &    0 &    0 &    0 &    0 &    0 &    0 &  0.010 &  0.000 &  485101.212 \\\\\n1 &   0 &   0 &   0 &   0 &   0 &   0 &   0 &   0 &   0 &    1 &    0 &    0 &    1 &    0 &    0 &    0 &  0.015 &  1.041 &       1.042 \\\\\n2 &   0 &   0 &   0 &   0 &   0 &   0 &   0 &   0 &   1 &    1 &    0 &    0 &    1 &    1 &    0 &    0 &  0.255 &  1.042 &       1.048 \\\\\n3 &   0 &   0 &   0 &   0 &   0 &   0 &   0 &   0 &   1 &    0 &    1 &    0 &    0 &    0 &    1 &    0 &  0.255 &  1.041 &       1.042 \\\\\n4 &   0 &   0 &   0 &   0 &   0 &   0 &   0 &   0 &   0 &    0 &    1 &    0 &    0 &    0 &    1 &    0 &  0.251 &  1.041 &       1.043 \\\\\n5 &   0 &   0 &   0 &   0 &   0 &   0 &   0 &   0 &   1 &    1 &    0 &    1 &

In [187]:
f = open("pure_nash_table.txt", "w")
f.write(table.to_latex())
f.close()


In [188]:
4 ** 2

16

In [69]:
2 ** 16

65536

In [70]:
4 ** 2

16

In [71]:
8 ** 2

64

In [74]:
2 ** 2, one_bit['player index'].unique()

(4, array([1, 3, 2, 4]))

In [75]:
4 ** 2, two_bit['player index'].unique()

(16, array([ 8,  9,  2,  3,  1,  4,  5,  7,  6, 11, 10, 12, 13, 16, 14, 15]))

In [76]:
8 ** 2, three_bit['player index'].unique()

(64,
 array([16, 17, 29, 15, 14, 28, 10, 38, 39, 11, 13, 12, 49, 61, 60, 48, 62,
         8,  9, 63, 64, 58, 59, 40, 54,  2,  3, 55, 41, 57, 43,  1, 42, 56,
        52, 46,  4,  5, 47, 53, 45, 51,  7,  6, 50, 44, 23, 37, 36, 22, 34,
        20, 21, 35, 31, 25, 19, 18, 24, 30, 26, 32, 33, 27]))

In [77]:
four_bit['player index'].unique()

array([ 58,  70,  64, 206,   1, 212, 158, 170, 164, 165, 171, 159, 213,
       207,  65,  71,  59,  98,  67,  73, 239, 211,   2, 205, 167, 173,
       198, 199, 172, 166, 204, 210,   3, 238,  72,  66,  99,  89,  62,
        76,   7, 214, 200, 228, 162, 176, 189, 188, 177, 163, 229, 201,
         6, 215,  77,  63,  88,  75,  61,  49, 203, 217,   4, 175, 161,
       149, 148, 160, 174, 216,   5, 202,  48,  60,  74,  13, 113, 107,
       106, 112,  12,  38,  10, 138, 104, 110, 111, 105, 139,  11,  39,
        15,  29, 101, 115, 129, 128, 114, 100,  28,  14,  16, 248, 116,
       102, 103, 117, 249,  17,  32,  26, 244, 250, 132, 126, 127, 133,
       251, 245,  27,  33,  19,  25,  31, 253, 247, 119, 125, 131, 130,
       124, 118, 246, 252,  30,  24,  18,  20,  34, 256, 242, 120, 134,
       108, 109, 135, 121, 243,  35,  21,  37,  23, 241, 255, 137, 123,
       122, 136, 254, 240,  22,  36,  92,  86,  79,  51,  45,   8, 227,
       233, 179, 151, 145, 192, 186, 187, 193, 144, 150, 178, 23

In [78]:
4 * 4 * 4 * 4

256

In [80]:
4 ** 2, 2 ** 4

(16, 16)

In [81]:
8 ** 2, 2 ** 8

(64, 256)